In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.path as path

## Plotting and contour helpers

In [ ]:
def linf_circle(x_c=0., y_c=0., r=1.):
    '''
    Parametric x, y points to represent a 'circle' in L^\infty
    '''
    x = [x_c-r, x_c-r, x_c+r, x_c+r, x_c-r]
    y = [y_c-r, y_c+r, y_c+r, y_c-r, y_c-r]
    return x, y

def linf_path(start, end):
    '''
    Parametric x, y points to represent a 'distance' path in L^\infty.

    Represented as a broken line connected to both points
    - parallel to the axis of greatest difference
    - line length represents distance
    '''
    startx, starty = start
    endx, endy = end
    if np.abs(startx - endx) > np.abs(starty - endy):
        midx = startx + (endx - startx)/2
        dy = (endy - starty)/20
        return [startx, midx, midx] + [None] + [midx, midx, endx], [starty, starty, starty+dy] + [None] + [endy-dy, endy, endy]
    else:
        midy = starty + (endy - starty)/2
        dx = (endx - startx)/20
        return [startx, startx, startx+dx] + [None] + [endx-dx, endx, endx], [starty, midy, midy] + [None] + [midy, midy, endy]
    
def l2_circle(x_c=0., y_c=0., r=1.):
    '''
    Parametric x, y points to represent a circle in L^2
    '''
    t = np.linspace(-np.pi, np.pi, 100)
    x = x_c + r * np.cos(t)
    y = y_c + r * np.sin(t)
    return x, y

def l2_path(start, end):
    '''
    Parametric x, y points to represent a distance path in L^2
    '''
    return list(zip(start, end))

def l1_circle(x_c=0., y_c=0., r=1.):
    '''
    Parametric x, y points to represent a 'circle' in L^1
    '''
    x = [x_c-r, x_c, x_c+r, x_c, x_c-r]
    y = [y_c, y_c+r, y_c, y_c-r, y_c]
    return x, y

def l1_path(start, end):
    '''
    Parametric x, y points to represent a distance path in L^2.
    
    Represented as a bent line taking each axis in turn
    '''
    return [start[0], start[0], end[0]], [start[1], end[1], end[1]]

In [ ]:
def distance_map(norm_split, point, minx=-1, maxx=1, miny=-1, maxy=1):
    '''
    Produce a grid map of norm distance from a given point
    :param norm_split: 'split' norm function taking xdiff, ydiff as parameters
    :param point: xy position of point to measure from
    '''
    xs = np.linspace(minx, maxx, 100)
    ys = np.linspace(miny, maxy, 100)
    px, py = point
    xx, yy = np.meshgrid(xs-px, ys-py, sparse=True)
    return norm_split(xx, yy)

def linf_split(xdists, ydists):
    return np.maximum(np.abs(xdists), np.abs(ydists))

def l2_split(xdists, ydists):
    return np.sqrt(xdists**2 + ydists**2)

def l1_split(xdists, ydists):
    return np.abs(xdists) + np.abs(ydists)

## Producing graphics for alignment metrics

### Mixing different distance/normalisation

In [ ]:
def plot_mixed_norms_opposites(start, other, circle_normalisation, path_distance, norm_distance, norm=None):
    '''
    A pretty illustration of using different norm for distance and normalisation
    :param start: identified point to measure distance from
    :param other: another point, not opposite, to illustrate distance
    :param circle_normalisation: function to produce circle representation for the normalisation norm
    :param path_distance: function to produce distance path representation for the distance norm
    :param norm_distance: function to measure the distance norm given diffx and diffy
    :param norm: function to measure the normalisation
    '''
    if norm is not None:
        start = start / norm(*start)
        other = other / norm(*other)

    unit_circle = circle_normalisation()
    ds = distance_map(norm_distance, start)

    opp_path = path_distance(start, -start)
    other_path = path_distance(start, other)

    fig, ax = plt.subplots()
    ax.plot(*unit_circle, '--', c='black', label='unit circle (normalisation)')
    im = ax.imshow(ds, extent=[-1, 1, -1, 1], origin='lower')
    clip = path.Path(np.stack(unit_circle, -1))
    im.set_clip_path(clip, transform=ax.transData)
    fig.colorbar(im, label='distance')

    ax.scatter(*start, c='black', label='true opposites')
    ax.scatter(*-start, c='black')
    ax.scatter(*other, c='red', label='further (distance)')

    ax.plot(*opp_path, ':', c='black')
    ax.plot(*other_path, ':', c='red')

    ax.set_xlim(-1.1, 1.1)
    ax.set_ylim(-1.1, 1.1)
    return fig, ax

A plot of the 2d $L^\infty$ standard surface (a square), with $L^2$ distances, showing

- a pair of opposite points having one distance
- a third, non-opposite point having _greater_ ($L^2$) distance

In [ ]:
_, ax = plot_mixed_norms_opposites(
    start=np.array([-1, 0]),
    other=np.array([1, -0.5]),
    circle_normalisation=linf_circle,
    path_distance=l2_path,
    norm_distance=l2_split)
ax.set_title('$L^2$ distance with $L^\infty$ normalisation')
ax.legend(loc='upper left');

A plot of the 2d $L^2$ standard surface (a circle), with $L^\infty$ distances, showing

- a pair of opposite points having one distance
- a third, non-opposite point having _greater_ ($L^\infty$) distance

In [ ]:
_, ax = plot_mixed_norms_opposites(
    start=np.array([-1, -1]),
    other=np.array([1, -0.05]),
    circle_normalisation=l2_circle,
    path_distance=linf_path,
    norm_distance=linf_split,
    norm=l2_split)
ax.set_title('$L^\infty$ distance with $L^2$ normalisation')
ax.legend(loc='upper left');